In [2]:
from __future__ import division
import os, sys
import urllib, cStringIO
import pymongo as pm ## first establish ssh tunnel to server where database is running
import base64
import numpy as np
from numpy import *
import PIL
from PIL import Image
import base64
import matplotlib
from matplotlib import pylab, mlab, pyplot
from IPython.core.pylabtools import figsize, getfigs
plt = pyplot
import seaborn as sns
sns.set_context('poster')
sns.set_style('white')
from matplotlib.path import Path
import matplotlib.patches as patches

import pandas as pd
from svgpathtools import parse_path, wsvg, svg2paths

/Users/megumisano/anaconda/envs/py27/lib/python2.7/site-packages/scipy/spatial/__init__.py:96: ImportWarning: Not importing directory '/Users/megumisano/anaconda/envs/py27/lib/python2.7/site-packages/scipy/spatial/qhull': missing __init__.py
  from .qhull import *
/Users/megumisano/anaconda/envs/py27/lib/python2.7/site-packages/scipy/optimize/_minimize.py:37: ImportWarning: Not importing directory '/Users/megumisano/anaconda/envs/py27/lib/python2.7/site-packages/scipy/optimize/lbfgsb': missing __init__.py
  from .lbfgsb import _minimize_lbfgsb


### setup

In [3]:
# directory & file hierarchy
proj_dir = os.path.abspath('../../..')
analysis_dir = os.getcwd()
results_dir = os.path.join(proj_dir,'results')
plot_dir = os.path.join(results_dir,'plots')
csv_dir = os.path.join(results_dir,'csv')
exp_dir = os.path.abspath(os.path.join(proj_dir,'experiments'))
sketch_dir = os.path.abspath(os.path.join(proj_dir,'sketches'))

## add helpers to python path
if os.path.join(proj_dir,'analysis','python') not in sys.path:
    sys.path.insert(1,os.path.join(proj_dir,'analysis','python'))
    
if not os.path.exists(results_dir):
    os.makedirs(results_dir)
    
if not os.path.exists(plot_dir):
    os.makedirs(plot_dir)   
    
if not os.path.exists(csv_dir):
    os.makedirs(csv_dir)       
    
# Assign variables within imported analysis helpers
import analysis_helpers as h
if sys.version_info[0]>=3:
    from importlib import reload
reload(h)

import svg_rendering_helpers as srh
reload(srh)

<module 'svg_rendering_helpers' from '/Users/megumisano/graphical_conventions/analysis/python/svg_rendering_helpers.pyc'>

In [4]:
# set vars 
auth = pd.read_csv('auth.txt', header = None) # this auth.txt file contains the password for the sketchloop user
pswd = auth.values[0][0]
user = 'sketchloop'
host = 'rxdhawkins.me' ## cocolab ip address

# have to fix this to be able to analyze from local
import pymongo as pm
conn = pm.MongoClient('mongodb://sketchloop:' + pswd + '@127.0.0.1')
db = conn['3dObjects']
coll = db['graphical_conventions']

# which iteration name should we use?
## "good" iterations that we currently combine: ['run3_size4_waiting', 'run4_generalization']
iterationNames = ['run3_size4_waiting', 'run4_generalization']

#### wsvg arguments
```
wsvg(paths=None, colors=None, filename=join, stroke_widths=None, nodes=None, node_colors=None, node_radii=None, openinbrowser=False, timestamp=False, margin_size=0.1, mindim=600, dimensions=None, viewbox=None, text=None, text_path=None, font_size=None, attributes=None, svg_attributes=None)
```

### what we need (tests on single sketches)

- render_svg
- svg_to_png
- svg2paths (already defined in svgpathtools)

In [13]:
### rendering stroke-by-stroke 
stroke_dir = os.path.join(sketch_dir, 'stroke_analysis')

In [103]:
reload(srh)
## render out svg
srh.render_svg(parsed,base_dir=stroke_dir)

In [7]:
## get svg path list
svg_paths = srh.generate_svg_path_list(os.path.join(stroke_dir,'svg'))

In [8]:
## convert to png
srh.svg_to_png(svg_paths,base_dir=stroke_dir)

convert /Users/megumisano/graphical_conventions/sketches/stroke_analysis/svg/4479-3f10bccf-ef14-4c6a-a2f5-333f692f78d8_6_waiting_01_0_run4_generalization_0_end.svg /Users/megumisano/graphical_conventions/sketches/stroke_analysis/png/4479-3f10bccf-ef14-4c6a-a2f5-333f692f78d8_6_waiting_01_0_run4_generalization_0_end.png


### svg rendering with stroke deletion in both directions

In [83]:
D = pd.read_csv(os.path.join(results_dir, 'graphical_conventions.csv'))

In [103]:
reload(srh)
for g in D['gameID].unique()':
    D_game = D[D['gameID'] == g]
    print "rendering sketches from game " + g + " with " + str(len(D_game)) + " trials"
    for t in range(1, 41):
        D_trial = D_game[D_game['trialNum'] == t]
        iterationName = 'run4_generalization' if list(D_trial['Generalization'])[0] == 'between' else 'run3_size4_waiting'
        trial_level_path = g + '_' + str(t) + '_' + list(D_trial['target'])[0] + '_' + str(list(D_trial['repetition'])[0]) + '_' + iterationName
        if int(trial_level_path.split('_')[1]) > 40:
            print(trial_level_path)
        svg_list = ast.literal_eval(D[(D['gameID'] == g) & (D['trialNum'] == t)]['svgString'].unique()[0])
        assert len(svg_list) != 0
        # deleting from the end 
        for num_strokes in range(len(svg_list)): # 0 to 7
            stroke_and_direction = '_' + str(num_strokes) + '_start.svg'  # deleting how many strokes? 
            stroke_level_path = trial_level_path + stroke_and_direction
            parsed = [parse_path(svg_list[i]) for i in range(num_strokes,len(svg_list))]
            ## render out svg
            srh.render_svg(parsed,base_dir=stroke_dir, out_fname=stroke_level_path)
        # and deleting from the start 
        for num_strokes in range(len(svg_list)): # 0 to 7  
            stroke_and_direction = '_' + str(num_strokes) + '_end.svg' # deleting how many strokes? 
            stroke_level_path = trial_level_path + stroke_and_direction
            parsed = [parse_path(svg_list[i]) for i in range(0, len(svg_list) - num_strokes)]
            ## render out svg
            srh.render_svg(parsed,base_dir=stroke_dir, out_fname=stroke_level_path)

rendering sketches from game 5408-c0bdf20b-7f65-49fb-aea3-0dd72e154ebd with 40 trials
rendering sketches from game 7940-e30c3a20-dd99-4efc-ae92-f650a7f1d6e6 with 40 trials
rendering sketches from game 9338-3d6dc953-6aca-48c1-b949-4e5491d3b967 with 40 trials


In [90]:
## get svg path list
svg_paths = srh.generate_svg_path_list(os.path.join(stroke_dir,'svg'))

In [101]:
## convert to png
reload(srh)
srh.svg_to_png(diff_paths_,base_dir=stroke_dir)

convert /Users/megumisano/graphical_conventions/sketches/stroke_analysis/svg/9817-6f2a66db-fc24-482d-aa20-1980ad6f524e_38_dining_02_7_run3_size4_waiting_5_start.svg /Users/megumisano/graphical_conventions/sketches/stroke_analysis/png2/9817-6f2a66db-fc24-482d-aa20-1980ad6f524e_38_dining_02_7_run3_size4_waiting_5_start.png
